In [1]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
# BLOQUE 1: imports y utilidades
from pathlib import Path
import pandas as pd
import numpy as np

from IPython.display import display, HTML
import ipywidgets as widgets

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)

def norm_emp_id(x):
    if pd.isna(x): return x
    s = str(x).strip()
    return s.zfill(4) if s.isdigit() and len(s) <= 4 else s

def fmt_pct(x):
    if pd.isna(x): 
        return "-"
    try:
        return f"{x:.1%}"
    except Exception:
        return "-"


In [ ]:
# BLOQUE 2: paths y lectura de trips
PATH_TRIPS = Path("data/processed/gps_match_trips.parquet")
PATH_POINTS = Path("data/processed/gps_match_points.parquet") 
PATH_EOTS = Path("data/raw/eots.csv")
PATH_RUTAS = Path("data/raw/catalogo_rutas_cid.csv")
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

trips = pd.read_parquet(PATH_TRIPS, engine="pyarrow").copy()

# Normalizaciones mínimas
for c in ("agency_id","route_id","ruta_hex","mean_id"):
    if c in trips.columns:
        trips[c] = trips[c].astype(str).str.upper().str.strip()
if "agency_id" in trips.columns:
    trips["agency_id"] = trips["agency_id"].apply(norm_emp_id)

# Asegurar numéricos
for c in ("pts_en_declared","pts_trip","ratio"):
    if c in trips.columns:
        trips[c] = pd.to_numeric(trips[c], errors="coerce")

# Columnas derivadas base
if "ratio" not in trips.columns and {"pts_en_declared","pts_trip"}.issubset(trips.columns):
    trips["ratio"] = trips["pts_en_declared"] / trips["pts_trip"]

trips["trip_match"] = trips["ratio"] >= 0.60
trips["route_match"] = (trips.get("route_id").astype(str) == trips.get("ruta_hex").astype(str))

trips["pts_in"]  = trips.get("pts_en_declared", pd.Series([np.nan]*len(trips)))
trips["pts_out"] = trips.get("pts_trip", pd.Series([np.nan]*len(trips))) - trips["pts_in"]

# Hora si existe
if "hora" in trips.columns:
    trips["hora"] = trips["hora"].astype("Int64")

print("shape:", trips.shape)
display(trips.head(3))


shape: (521, 14)


,agency_id,mean_id,trip_id,route_id,ruta_hex,pts_en_declared,pts_trip,ratio,trip_match,route_id_match,hora,route_match,pts_in,pts_out
0,0005,005DD,-1,001D,NONE,0,77,0.0,False,False,10,False,0,77
1,0005,005DD,0,001D,00B1,0,110,0.0,False,False,10,False,0,110
2,0005,005DF,0,001D,008C,0,196,0.0,False,False,10,False,0,196


In [4]:
# BLOQUE 3: construir KPIs por empresa + ruta + hora
group_cols = [c for c in ["agency_id","ruta_hex","hora"] if c in trips.columns]
if not group_cols:
    group_cols = ["agency_id","ruta_hex"]  # fallback

facts = (
    trips.groupby(group_cols, dropna=False)
         .agg(
             total_trips=("trip_id","nunique"),
             trips_ok=("trip_match","sum"),
             trips_route_match=("route_match","sum"),
             total_pts_in=("pts_in","sum"),
             total_pts_out=("pts_out","sum")
         )
         .reset_index()
)

# Porcentajes
facts["pct_trips_ok"]    = (facts["trips_ok"] / facts["total_trips"]).replace([np.inf,-np.inf], np.nan)
facts["pct_route_match"] = (facts["trips_route_match"] / facts["total_trips"]).replace([np.inf,-np.inf], np.nan)
facts["pct_pts_in"]      = (facts["total_pts_in"] / (facts["total_pts_in"] + facts["total_pts_out"])).replace([np.inf,-np.inf], np.nan)

# Redondeos
for c in ["pct_trips_ok","pct_route_match","pct_pts_in"]:
    if c in facts.columns:
        facts[c] = facts[c].round(3)

print("KPIs construidos:")
display(facts.sort_values("total_trips", ascending=False).head(10))


KPIs construidos:


,agency_id,ruta_hex,hora,total_trips,trips_ok,trips_route_match,total_pts_in,total_pts_out,pct_trips_ok,pct_route_match,pct_pts_in
3,0005,0055,10,2,4,0,596,0,2.0,0.0,1.000
4,0005,008C,10,2,23,0,3765,3061,11.5,0.0,0.552
5,0005,00B1,10,2,0,0,0,424,0.0,0.0,0.000
15,0007,00E6,10,2,6,0,635,1182,3.0,0.0,0.349
12,0007,003A,10,2,10,0,1272,370,5.0,0.0,0.775
13,0007,0051,10,2,0,0,55,851,0.0,0.0,0.061
14,0007,008C,10,2,20,0,3469,5794,10.0,0.0,0.375
8,0005,011E,10,2,11,0,1823,22,5.5,0.0,0.988
56,0020,0108,10,2,2,0,162,2275,1.0,0.0,0.066
57,0020,010B,10,2,8,0,541,1532,4.0,0.0,0.261


In [5]:
# BLOQUE 4: consistencia route_id vs ruta_hex
have_route_cols = {"route_id","ruta_hex"}.issubset(trips.columns)
if have_route_cols:
    pairs = (
        trips.groupby(["agency_id","route_id","ruta_hex"], dropna=False)
             .agg(total_trips=("trip_id","nunique"),
                  trips_ok=("trip_match","sum"))
             .reset_index()
             .sort_values("total_trips", ascending=False)
    )
    mismatch = pairs[pairs["route_id"] != pairs["ruta_hex"]]
    print("Top 15 combinaciones declarada≠ejecutada:")
    display(mismatch.head(15))
else:
    pairs = pd.DataFrame()
    print("No hay columnas 'route_id' y/o 'ruta_hex' para la tabla de consistencia.")


Top 15 combinaciones declarada≠ejecutada:


,agency_id,route_id,ruta_hex,total_trips,trips_ok
246,0022,0110,008C,2,7
122,0007,016C,008C,2,0
5,0005,001E,008C,2,0
110,0007,0098,00E6,2,3
1,0005,001D,008C,2,0
87,0007,0045,0051,2,0
50,0007,0036,008C,2,3
108,0007,0098,003A,2,2
27,0005,0027,011E,2,4
52,0007,0036,00EE,2,2


In [6]:
len(trips), trips[["agency_id","mean_id","trip_id"]].drop_duplicates().shape


(521, (521, 3))

In [7]:
# BLOQUE 5: filtros interactivos (ipywidgets) + métricas + tablas

# Dimensiones auxiliares (si tenés eots.csv para mostrar nombres)
try:
    eots = pd.read_csv(PATH_EOTS, dtype=str)
    # buscar columnas posibles
    col_id_emp = None
    for c in eots.columns:
        if any(key in c.lower() for key in ["eot_id","cod_catalogo","agency","id_eot_vmt_hex"]):
            col_id_emp = c; break
    col_nom_emp = None
    for c in eots.columns:
        if any(key in c.lower() for key in ["nombre","permisionario","razon"]):
            col_nom_emp = c; break

    if col_id_emp:
        eots["agency_id"] = eots[col_id_emp].astype(str).apply(norm_emp_id)
    if col_nom_emp:
        eots["empresa_nombre"] = eots[col_nom_emp].astype(str)

    dim_emp = eots[["agency_id","empresa_nombre"]].dropna().drop_duplicates() if {"agency_id","empresa_nombre"}.issubset(eots.columns) else None
except Exception:
    dim_emp = None

facts_view = facts.copy()
if dim_emp is not None:
    facts_view = facts_view.merge(dim_emp, on="agency_id", how="left")

# Widgets
emp_col = "empresa_nombre" if ("empresa_nombre" in facts_view.columns and facts_view["empresa_nombre"].notna().any()) else "agency_id"

empresas = sorted(facts_view[emp_col].dropna().unique().tolist())
rutas_exec = sorted(facts_view["ruta_hex"].dropna().unique().tolist()) if "ruta_hex" in facts_view.columns else []
horas = sorted([int(h) for h in facts_view["hora"].dropna().unique().tolist()]) if "hora" in facts_view.columns else []

w_emp = widgets.SelectMultiple(options=empresas, description="Empresa", rows=6)
w_ruta = widgets.SelectMultiple(options=rutas_exec, description="Ruta (hex)", rows=8)
w_hora = widgets.SelectMultiple(options=horas, description="Hora", rows=6)

btn_export = widgets.Button(description="Exportar selección (CSV)", button_style='')

out = widgets.Output()

def apply_filters(*args):
    with out:
        out.clear_output()
        df = facts_view.copy()
        # filtros
        if w_emp.value:
            df = df[df[emp_col].isin(list(w_emp.value))]
        if "ruta_hex" in df.columns and w_ruta.value:
            df = df[df["ruta_hex"].isin(list(w_ruta.value))]
        if "hora" in df.columns and w_hora.value:
            df = df[df["hora"].isin(list(w_hora.value))]

        # métricas
        total_trips = int(df["total_trips"].sum()) if not df.empty else 0
        trips_ok = int(df["trips_ok"].sum()) if "trips_ok" in df.columns and not df.empty else 0
        trips_rm = int(df["trips_route_match"].sum()) if "trips_route_match" in df.columns and not df.empty else 0
        pts_in = int(df["total_pts_in"].sum()) if "total_pts_in" in df.columns and not df.empty else 0
        pts_out = int(df["total_pts_out"].sum()) if "total_pts_out" in df.columns and not df.empty else 0

        print("=== KPIs de la selección ===")
        print(f"- Total trips: {total_trips:,}")
        print(f"- Trips OK (ratio ≥0.60): {trips_ok:,}   ({fmt_pct(trips_ok / total_trips if total_trips else np.nan)})")
        print(f"- Route ID coincide (trips): {trips_rm:,}   ({fmt_pct(trips_rm / total_trips if total_trips else np.nan)})")
        print(f"- Puntos dentro de ruta: {pts_in:,}   ({fmt_pct(pts_in / (pts_in + pts_out) if (pts_in + pts_out) else np.nan)})")
        print(f"- Puntos fuera de ruta:  {pts_out:,}   ({fmt_pct(pts_out / (pts_in + pts_out) if (pts_in + pts_out) else np.nan)})")

        # tabla detalle ordenada
        cols_show = [c for c in [emp_col,"agency_id","ruta_hex","hora","total_trips","trips_ok","pct_trips_ok","trips_route_match","pct_route_match","total_pts_in","total_pts_out","pct_pts_in"] if c in df.columns]
        if not df.empty and cols_show:
            display(df[cols_show].sort_values(["agency_id","ruta_hex","hora"], na_position="last").reset_index(drop=True))
        else:
            print("\n(No hay filas para la selección actual)")

        # si existe pairs (declarada vs ejecutada), mostrar un resumen para la selección de empresas/rutas/horas
        if not pairs.empty:
            sel_ag = df["agency_id"].unique().tolist() if not df.empty else facts_view["agency_id"].unique().tolist()
            sel_rh = df["ruta_hex"].unique().tolist() if ("ruta_hex" in df.columns and not df.empty) else None

            p = pairs.copy()
            p = p[p["agency_id"].isin(sel_ag)]
            if sel_rh:
                p = p[p["ruta_hex"].isin(sel_rh)]

            if not p.empty:
                print("\n--- Declarada vs Ejecutada (selección) ---")
                # indicador global
                total_pairs_trips = int(p["total_trips"].sum())
                exact_pairs = int(p.loc[p["route_id"] == p["ruta_hex"], "total_trips"].sum())
                print(f"Coincidencia declarada=ejecutada (trips): {exact_pairs:,} / {total_pairs_trips:,}   ({fmt_pct(exact_pairs/total_pairs_trips if total_pairs_trips else np.nan)})")

                display(p.sort_values("total_trips", ascending=False).head(20))

def export_csv(btn):
    df = facts_view.copy()
    if w_emp.value:
        df = df[df[emp_col].isin(list(w_emp.value))]
    if "ruta_hex" in df.columns and w_ruta.value:
        df = df[df["ruta_hex"].isin(list(w_ruta.value))]
    if "hora" in df.columns and w_hora.value:
        df = df[df["hora"].isin(list(w_hora.value))]
    out_path = OUT_DIR / "kpi_selection.csv"
    df.to_csv(out_path, index=False)
    with out:
        print(f"\n✅ Exportado: {out_path.resolve()}")

w_emp.observe(apply_filters, names="value")
w_ruta.observe(apply_filters, names="value")
w_hora.observe(apply_filters, names="value")
btn_export.on_click(export_csv)

display(widgets.HBox([w_emp, w_ruta, w_hora], layout=widgets.Layout(justify_content='space-between')))
display(btn_export)
display(out)

# primera render
apply_filters()


Button(description='Exportar selección (CSV)', style=ButtonStyle())

Output()

In [ ]:
# BLOQUE 6 : KPI por bus (empresa + ruta + hora + mean_id)
group_bus = [c for c in ["agency_id","ruta_hex","hora","mean_id"] if c in trips.columns]
if group_bus:
    facts_bus = (
        trips.groupby(group_bus, dropna=False)
             .agg(total_trips=("trip_id","nunique"),
                  trips_ok=("trip_match","sum"),
                  total_pts_in=("pts_in","sum"),
                  total_pts_out=("pts_out","sum"))
             .reset_index()
    )
    facts_bus["pct_trips_ok"] = (facts_bus["trips_ok"] / facts_bus["total_trips"]).replace([np.inf,-np.inf], np.nan).round(3)
    facts_bus["pct_pts_in"]   = (facts_bus["total_pts_in"] / (facts_bus["total_pts_in"] + facts_bus["total_pts_out"])).replace([np.inf,-np.inf], np.nan).round(3)

    print("Peores buses por % trips OK:")
    display(facts_bus.sort_values("pct_trips_ok", ascending=True).head(15))
else:
    print("No hay columnas suficientes para KPI por bus.")


Peores buses por % trips OK:


,agency_id,ruta_hex,hora,mean_id,total_trips,trips_ok,total_pts_in,total_pts_out,pct_trips_ok,pct_pts_in
29,0005,008C,10,005F7,1,0,52,130,0.0,0.286
467,0025,011E,10,00332,1,0,0,176,0.0,0.000
468,0025,011E,10,004B7,1,0,0,184,0.0,0.000
472,0025,011E,10,00527,1,0,0,185,0.0,0.000
473,0025,011E,10,0052C,1,0,0,16,0.0,0.000
474,0025,011E,10,0052D,1,0,0,184,0.0,0.000
0,0005,001E,10,005E3,1,0,0,44,0.0,0.000
318,0017,01BB,10,007EA,1,0,0,200,0.0,0.000
319,0017,01BB,10,007ED,1,0,0,100,0.0,0.000
288,0017,008C,10,007F0,1,0,0,182,0.0,0.000


In [ ]:
# BLOQUE 7: guardar KPIs para uso posterior
facts.to_parquet(OUT_DIR / "kpi_facts_notebook.parquet", engine="pyarrow", index=False)
facts.to_csv(OUT_DIR / "kpi_facts_notebook.csv", index=False)
if not pairs.empty:
    pairs.to_parquet(OUT_DIR / "kpi_pairs_decl_vs_exec.parquet", engine="pyarrow", index=False)
    pairs.to_csv(OUT_DIR / "kpi_pairs_decl_vs_exec.csv", index=False)

print(" Guardado:")
print(" -", (OUT_DIR / "kpi_facts_notebook.parquet").resolve())
print(" -", (OUT_DIR / "kpi_facts_notebook.csv").resolve())
if not pairs.empty:
    print(" -", (OUT_DIR / "kpi_pairs_decl_vs_exec.parquet").resolve())
    print(" -", (OUT_DIR / "kpi_pairs_decl_vs_exec.csv").resolve())


✅ Guardado:
 - C:\Users\Usuario\Desktop\Python\cumplimiento-rutas-h3\data\processed\kpi_facts_notebook.parquet
 - C:\Users\Usuario\Desktop\Python\cumplimiento-rutas-h3\data\processed\kpi_facts_notebook.csv
 - C:\Users\Usuario\Desktop\Python\cumplimiento-rutas-h3\data\processed\kpi_pairs_decl_vs_exec.parquet
 - C:\Users\Usuario\Desktop\Python\cumplimiento-rutas-h3\data\processed\kpi_pairs_decl_vs_exec.csv
